In [117]:
import pandas as pd

df = pd.read_csv('..\\zipcode/MASTER_ORGANIZATION_NT_MAPPING_20230601(แก้ไข).csv', dtype=str)

zipcode = pd.read_excel('..\\zipcode/zipcode_wikipedia_.xlsx', dtype=str)
province = pd.read_csv('..\\zipcode/romanized_province.csv', dtype=str)
province['thai_province'] = province['thai_province'].str.strip()

จัดเตรียมชื่อจังหวัด, ไปรษณีย์ไทย

In [118]:
zipcode['อำเภอ'] = zipcode['อำเภอ'].fillna(zipcode['เขต'])
zipcode.loc[len(zipcode)] = ['', '10210', '', 'กรุงเทพมหานคร']

In [119]:
zipcode['อำเภอ'] = zipcode['อำเภอ'].str.replace(r'^เมือง', '',regex=True)
zipcode.rename(columns={'อำเภอ': 'thai_province'}, inplace=True)
zipcode = pd.merge(province, zipcode, how='left', on='thai_province')
zipcode = zipcode.loc[(zipcode['รหัสไปรษณีย์'].str.contains(r'10270|10210|\d\d000', na=False)) & (~zipcode['province_id'].isna())]
zipcode = zipcode.drop(columns=['เขต', 'หมายเหตุ', 'province_id'])
zipcode['romanized_province'] = zipcode['romanized_province'].str.title()

In [120]:
def update_province_info(row, zipcode):
    for _, zipcode_row in zipcode.iterrows():
        if zipcode_row['thai_province'] in row['ส่วน']:
            row['PROVINCE'] = zipcode_row['romanized_province']
            row['PROVINCE (THAI)'] = zipcode_row['thai_province']
            row['ZIPCODE'] = zipcode_row['รหัสไปรษณีย์']
            break
    return row

df = df.apply(update_province_info, axis=1, zipcode=zipcode)

In [121]:
df.to_csv('D:/student/Peace/test/zipcode/MASTER_ORGANIZATION_NT_MAPPING_20230601(แก้ไขแล้ว).csv', index=False)